In [1]:
import pickle
import pandas as pd
import numpy as np

import random
from itertools import product

## 옳은 문장 데이터 셋 구성

### 자기소개서 데이터 Load

In [2]:
f = open("JobKorea_split_content.pickle", "rb")
processed_contents = pickle.load(f)
f.close()

processed_contents.head(2)

,company_name,company_url,apply_info,applicant_info,score,score_desc,url,Question,content,comment,split_content
0,(주)경동나비엔,https://www.jobkorea.co.kr/Recruit/Co_Read/C/k...,2021년 상반기 신입 제품·서비스영업,수도권4년 중어중문학과 학점 4.26/4.5 토익 845 제2외국어 1개 자격증 ...,2.0,전반적으로 아쉬운 자기소개서입니다. 질문에 대한 명쾌한 답이 없다는 점이 본 자기소...,https://www.jobkorea.co.kr/starter/PassAssay/V...,[자신의 가장 자랑스러운 성취 경험 또는 자신을 가장 힘들게 했던 실패 경험은 무엇...,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...,[ 아쉬운점 1 지원자가 무슨 말을 하고자 하는지는 잘 알겠습니다. 하지만 이 글에...,"[[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다., 조금 늦은 나이..."
1,(주)포스코인터내셔널,https://www.jobkorea.co.kr/Recruit/Co_Read/C/e...,2021년 상반기 신입 수출입·무역사무,"고졸 자격증 5개 2,282읽음",2.0,일반적으로 많은 지원자들이 실수하는 방식대로 쓴 자기소개서입니다. 자신에 대해 소개...,https://www.jobkorea.co.kr/starter/PassAssay/V...,"[성장과정 및 생활신조 등 자기소개를 기술하여 주시기 바랍니다. , 성격, 능력 관...",[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도전...,[ 아쉬운점 1 이 항목은 이런 내용을 쓰는 항목이 아니라 지원자의 성장과정을 설명...,[[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도...


### 앞뒤문장으로 데이터셋 구성

In [4]:
def correctLabeling(contentsList):
    correctDataset = []
    
    for content in contentsList:
        for i in range(len(content[:-1])):
            correctDataset.append((content[i], content[i+1]))
            
    return correctDataset

In [5]:
correctLabel = []

for contents in processed_contents['split_content']:
    correctLabel.extend(correctLabeling(contents))
    
correctLabel = np.array(correctLabel)
correctLabel.shape

(231949, 2)

In [6]:
correctLabel[:5]

array([['새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다.',
        '조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.'],
       ['조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.',
        '잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.'],
       ['잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.',
        '저는 좌절보다는 노력을 택하였습니다.'],
       ['저는 좌절보다는 노력을 택하였습니다.',
        '매주 영어 학원과 스터디그룹에 참석하였으며 다양한 이들과의 대화를 위해 혼자 펍에 가 친구를 사귀기도 하고 아르바이트를 통해 많은 이들과의 소통으로 영어 실력뿐만 아니라 소통력과 관계력까지도 키울 수 있었습니다.'],
       ['매주 영어 학원과 스터디그룹에 참석하였으며 다양한 이들과의 대화를 위해 혼자 펍에 가 친구를 사귀기도 하고 아르바이트를 통해 많은 이들과의 소통으로 영어 실력뿐만 아니라 소통력과 관계력까지도 키울 수 있었습니다.',
        '이러한 경험을 바탕으로 경동나비엔의 일원이 되어서도 어떠한 어려움 속에서도 좌절하지 않고 백절불굴의 자세로 부족한 점을 계속해서 연구하고 개선하여 끊임없는 변화 속에서 끊임없는 노력으로 성공이라는 꽃을 피우도록 하겠습니다.']],
      dtype='<U2420')

### 라벨링 값 입력

In [7]:
label = np.array([1]*len(correctLabel))
label = label.reshape(-1,1)
label.shape

(231949, 1)

In [8]:
final = np.hstack((correctLabel, label))
final[:5]

array([['새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다.',
        '조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.', '1'],
       ['조금 늦은 나이의 도전이라 부족한 영어 실력으로 많은 어려움에 부딪쳤습니다.',
        '잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.',
        '1'],
       ['잠깐 좌절도 했지만 그보다 가장 힘들었던 것은 부모님이 주신 소중한 기회를 제대로 활용하고 있지 못하고 있는 저의 모습이었습니다.',
        '저는 좌절보다는 노력을 택하였습니다.', '1'],
       ['저는 좌절보다는 노력을 택하였습니다.',
        '매주 영어 학원과 스터디그룹에 참석하였으며 다양한 이들과의 대화를 위해 혼자 펍에 가 친구를 사귀기도 하고 아르바이트를 통해 많은 이들과의 소통으로 영어 실력뿐만 아니라 소통력과 관계력까지도 키울 수 있었습니다.',
        '1'],
       ['매주 영어 학원과 스터디그룹에 참석하였으며 다양한 이들과의 대화를 위해 혼자 펍에 가 친구를 사귀기도 하고 아르바이트를 통해 많은 이들과의 소통으로 영어 실력뿐만 아니라 소통력과 관계력까지도 키울 수 있었습니다.',
        '이러한 경험을 바탕으로 경동나비엔의 일원이 되어서도 어떠한 어려움 속에서도 좌절하지 않고 백절불굴의 자세로 부족한 점을 계속해서 연구하고 개선하여 끊임없는 변화 속에서 끊임없는 노력으로 성공이라는 꽃을 피우도록 하겠습니다.',
        '1']], dtype='<U2420')

### 데이터 저장

In [34]:
f=open("Correctdata.pickle", 'wb')
pickle.dump(final, f)
f.close()

## 틀린 문장 데이터 셋

* 옳은 문장 데이터 총 개수 : 231949개
* 틀리 문장 데이터 총 개수 : 231949개로 구성해야함 - 옳은 문장 데이터와 같이
    * 다른 자소서의 랜덤 문장과 매치 : 77317
    * 한국어 대화 문장에서 질문과 매치 : 77316
    * 한국어 대화 문장에서 답변과 매치 : 77316

### 1개의 자기소개서의 문장 리스트로 생성

In [8]:
sentencesOfDoc = []
for contents in processed_contents['split_content']:
    temp = []
    
    for content in contents:
        temp.extend(content)
        
    sentencesOfDoc.append(temp)
len(sentencesOfDoc)

6631

### 외부데이터에서 문장 가져오기

In [9]:
dialog_1 = pd.read_excel('data/Korean_dialog_data/academy(4,773).xlsx', engine='openpyxl')
dialog_2 = pd.read_excel('data/Korean_dialog_data/accommodation(7,113).xlsx', engine='openpyxl')
dialog_3 = pd.read_excel('data/Korean_dialog_data/cafe(7,859).xlsx', engine='openpyxl')
dialog_4 = pd.read_excel('data/Korean_dialog_data/closet(15,826).xlsx', engine='openpyxl')
dialog_5 = pd.read_excel('data/Korean_dialog_data/lifeService(11,087).xlsx', engine='openpyxl')
dialog_6 = pd.read_excel('data/Korean_dialog_data/realEstate(8,131).xlsx', engine='openpyxl')
dialog_7 = pd.read_excel('data/Korean_dialog_data/Restaurant(15,726).xlsx', engine='openpyxl')
dialog_8 = pd.read_excel('data/Korean_dialog_data/retailStore(14,949).xlsx', engine='openpyxl')
dialog_9 = pd.read_excel('data/Korean_dialog_data/tourLeisureEntertainment(4,949).xlsx', engine='openpyxl')

dialogs = [dialog_1, dialog_2, dialog_3, dialog_4, dialog_5, dialog_6, dialog_7, dialog_8, dialog_9]

In [10]:
dialog_9.head(2)

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,1시간에 얼마인가요?,H,관광여가오락,pc방,1,1,이용 가격 문의,NaN,Q,NaN,1시간에 얼마인가요?,NaN,NaN,NaN,1시간,NaN,1 시간/이용시간
1,점원,처음 1시간은 1000원이고 이후 1시간은 500원씩 추가됩니다,H,관광여가오락,pc방,0,2,이용 가격 문의,NaN,A,NaN,NaN,NaN,NaN,처음 1시간은 1000원이고 이후 1시간은 500원씩 추가됩니다,"처음, 1시간, 1000원, 이후, 1시간, 500원, 추가",NaN,"1시간/이용시간, 1000원/이용가격, 1시간/이용시간, 500원/이용가격"


In [11]:
Qdata = []
Adata = []

for i, dialog in enumerate(dialogs):
    for sentence in dialog[dialog.QA == 'Q']['SENTENCE']:
        Qdata.append(sentence)
        
    for sentence in dialog[dialog.QA == 'A']['SENTENCE']:
        Adata.append(sentence)

In [12]:
len(Qdata), len(Adata)

(51065, 39348)

### 자소서 문장과 어색한 문장 pair Dataset 만들기

In [14]:
########################################################
######### 중복이 너무 많아 수행 시간이 오래걸림 #########
########################################################
# # 질문 매칭 - 한국어 대화 질문 문장 매칭
# incorrectQLabeling = []

# while len(incorrectQLabeling) < 77316:
#     while len(incorrectQLabeling) < 77316:
#         DocRandNum = random.randrange(len(sentencesOfDoc))
#         sentenceRandNum = random.randrange(len(sentencesOfDoc[DocRandNum]))
#         QusRandNum = random.randrange(len(Qdata))

#         setence = sentencesOfDoc[DocRandNum][sentenceRandNum]
#         nextSentence = Qdata[QusRandNum]

#         incorrectQLabeling.append((sentence, nextSentence))
#         print("현재 데이터 쌍 개수: {}".format(len(incorrectQLabeling)), end='\r', flush=True)
        
#     incorrectQLabeling = list(set(incorrectQLabeling))
#     print("현재 데이터 쌍 개수: {}".format(len(incorrectQLabeling)), end='\r', flush=True)
    
# # 질문 매칭 - 한국어 대화 답변 문장 매칭
# incorrectALabeling = []

# while len(incorrectQLabeling) < 77316:
#     while len(incorrectQLabeling) < 77316:
#         DocRandNum = random.randrange(len(sentencesOfDoc))
#         sentenceRandNum = random.randrange(len(sentencesOfDoc[DocRandNum]))
#         AnsRandNum = random.randrange(len(Adata))

#         setence = sentencesOfDoc[DocRandNum][sentenceRandNum]
#         nextSentence = Adata[AnsRandNum]

#         incorrectALabeling.append((sentence, nextSentence))
#         print("현재 데이터 쌍 개수: {}".format(len(incorrectALabeling)), end='\r', flush=True)
        
#     incorrectALabeling = list(set(incorrectALabeling))
#     print("현재 데이터 쌍 개수: {}".format(len(incorrectALabeling)), end='\r', flush=True)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



이전에 계속해서 랜덤 숫자를 입력하여 할 경우 중복되는 경우의 수가 자주 발생하여 **시간이 오래**걸린다

**대안**\
**조합**을 이용하여 자기소개서와 한국어 대화 질문 or 답변 문장의 모든 경우의 수를 구한뒤 원하는 개수만큼 랜덤 추출을 한다.\
실행 중간 memory부족이 발생하여 데이터 나누어서 실행

#### 자소서 문서와 한국어 대화 질문or답변 문장 pair index 데이터 구축

* 조합으로 경우의수를 찾고 랜덤으로 원하는 만큼의 조합 경우의 수 추출

In [13]:
Doc_idxList = list(range(len(sentencesOfDoc)))
Q_idxList = list(range(len(Qdata)))
DocQ_li = [Doc_idxList, Q_idxList]
DocQCombi = list(product(*DocQ_li))
len(DocQCombi)

338612015

In [16]:
randDocQ = []
while len(randDocQ) < 77316:
    idx = random.randrange(len(DocQCombi))
    if idx not in randDocQ: randDocQ.append(idx)
    print("현재 데이터 쌍 개수: {}".format(len(randDocQ)), end='\r', flush=True)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

현재 데이터 쌍 개수: 43526

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [18]:
DocQPairIdx = [DocQCombi[i] for i in randDocQ]
len(DocQPairIdx)

77316

In [19]:
f=open("DocQPairIdx.pickle", 'wb')
pickle.dump(DocQPairIdx, f)
f.close()

In [14]:
Doc_idxList = list(range(len(sentencesOfDoc)))
A_idxList = list(range(len(Adata)))
DocA_li = [Doc_idxList, A_idxList]
DocACombi = list(product(*DocA_li))
len(DocACombi)

260916588

In [15]:
randDocA = []
while len(randDocA) < 77316:
    idx = random.randrange(len(DocACombi))
    if idx not in randDocA: randDocA.append(idx)
    print("현재 데이터 쌍 개수: {}".format(len(randDocA)), end='\r', flush=True)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [16]:
DocAPairIdx = [DocACombi[i] for i in randDocA]
len(DocAPairIdx)

77316

In [17]:
f=open("DocAPairIdx.pickle", 'wb')
pickle.dump(DocAPairIdx, f)
f.close()

#### 자소서 문장과 한국어 대화 질문or답변 문장 pair Dataset 구축

In [21]:
f=open("DocQPairIdx.pickle", 'rb')
DocQPairIdx = pickle.load(f)
f.close()

f=open("DocAPairIdx.pickle", 'rb')
DocAPairIdx = pickle.load(f)
f.close()

In [28]:
inCorrectQdata = []

for DocIdx, QIdx in DocQPairIdx:
    sentenceRandIdx = random.randrange(len(sentencesOfDoc[DocIdx]))
    
    sentence = sentencesOfDoc[DocIdx][sentenceRandIdx]
    nextSentence = Qdata[QIdx]
    inCorrectQdata.append((sentence, nextSentence))

inCorrectAdata = []

for DocIdx, AIdx in DocAPairIdx:
    sentenceRandIdx = random.randrange(len(sentencesOfDoc[DocIdx]))
    
    sentence = sentencesOfDoc[DocIdx][sentenceRandIdx]
    nextSentence = Adata[AIdx]
    inCorrectAdata.append((sentence, nextSentence))

### 데이터 저장

In [29]:
# 리스트를 np 배열로 변경
inCorrectQdata = np.array(inCorrectQdata)
inCorrectAdata = np.array(inCorrectAdata)

# 옳지않은 문장 0으로 라벨링 
negativelabelQ = np.array([0]*len(inCorrectQdata))
negativelabelA = np.array([0]*len(inCorrectQdata))
negativelabelQ = negativelabelQ.reshape(-1,1)
negativelabelA = negativelabelA.reshape(-1,1)
negativelabelQ.shape, negativelabelA.shape

inCorrectQdata = np.hstack([inCorrectQdata, negativelabelQ])
inCorrectAdata = np.hstack([inCorrectAdata, negativelabelA])

# 결과 확인
inCorrectQdata.shape, inCorrectQdata[-102]

((77316, 3),
 array(['업무 이외의 시간을 투자함으로써 저는 입사 후 1년 내로 협상전문가 자격증을 취득하겠습니다.', '각각 얼마인가요?',
        '0'], dtype='<U2420'))

In [19]:
f=open("inCorrectQdata.pickle", 'wb')
pickle.dump(inCorrectQdata, f)
f.close()
f=open("inCorrectAdata.pickle", 'wb')
pickle.dump(inCorrectAdata, f)
f.close()

## 기타